In [61]:
import pandas as pd
import numpy as np
import operator

In [62]:
df = pd.read_csv(open('../data/df_million.csv'), low_memory=False)
df = df[df['text'].str.contains("00leave")== False]
df = df.drop('Unnamed: 0', 1)
print('有{}筆對話'.format(len(df)))


有1000000筆對話


In [20]:
df.columns

Index(['from', 'to', 'text', 'room', 'time'], dtype='object')

In [21]:
df = df.reset_index(drop=True)

In [22]:
df.head()

,from,to,text,room,time
0,ud4097fbb4471c8c54512814bf0182e03,u84fa1e46e99df373e0f978c2ba274f9a,wd,8aee77a7-1d90-4d0f-b3c3-3f7b94a90e36,1.471640e+12
1,ud4097fbb4471c8c54512814bf0182e03,u96e826dccd3c9db56d684a5ddffc3154,嗨嗨,fe214cfc-22d1-4736-92e7-d133190142b8,1.471643e+12
2,u96e826dccd3c9db56d684a5ddffc3154,ud4097fbb4471c8c54512814bf0182e03,蛤？,fe214cfc-22d1-4736-92e7-d133190142b8,1.471643e+12
3,ud4097fbb4471c8c54512814bf0182e03,u96e826dccd3c9db56d684a5ddffc3154,你是第二個使用者ＱＱ,fe214cfc-22d1-4736-92e7-d133190142b8,1.471643e+12
4,u96e826dccd3c9db56d684a5ddffc3154,ud4097fbb4471c8c54512814bf0182e03,哈哈哈哈哈哈哈,fe214cfc-22d1-4736-92e7-d133190142b8,1.471643e+12


## Groupby by 'from' column 

In [23]:
grouped_data = df.groupby('from')['text'].apply(list).reset_index()
len(grouped_data)

12691

In [28]:
grouped_data.head()
type(grouped_data.text[0])

list

In [9]:
print('id為 {} 的使用者傳送的訊息 ：{}'.format(grouped_data.iloc[0][0],grouped_data.iloc[0][1]))

id為 1000080793444348 的使用者傳送的訊息 ：['嗨嗨', '安安', '幾歲呢', '我也是', '😂😂', '我男生', '嗨嗨', '我男生😂😂', '嗨嗨', '欸欸不在嗎？', '嗨嗨', '？？', '安安', '男17', '喔喔ok', '😂😂', '嗯嗯', '對', '哈哈😂😂', '一樣吧', '哪裡人呀？', '哇嗚唯一不靠海', '嗨嗨', '我高二男生喔😂😂', '哈哈', '掰掰', '嗨嗨', '有人嗎？', '嗨嗨', '嗨嗨', '什麼？？', '😂😂', '我高二男生@@', '不好玩', '換了班換了人', '二類', 'XD', '怎麼？？', '過來人？', '我還有頭髮的', '😃😃', '噗ㄘ', '不是和尚啦', '不是', '嗨嗨', '安安', '高二男生喔😂😂', '哈哈', '那掰掰', '自己說', '噗ㄘ', '掰掰', '？？', '欸欸？', '安安', '高二男生喔😂😂', '……', '好喔', '嗨嗨', '你好', '我是男生', '啊啊好巧喔', 'XD', '欸欸？', '😶😶😶', '？', '嗯嗯', '加油', '嗨嗨', '那掰掰吧😂', '嗨嗨', '嗨嗨', '男生😶', '我？', '我說我男生', '嗨嗨', '我男生😶', '覺得這裡有點像woo了😂😂', 'Wootalk阿跟這個差不多', '不過裡面有90%的機率是男的', '好無聊']


## set stop word

In [10]:
import jieba
import jieba.posseg as pseg
import jieba.analyse

jieba.set_dictionary('jieba_txt/dict.txt.big.txt')
jieba.load_userdict("jieba_txt/userdict.txt")
stopwordset = set()
with open('jieba_txt/stop_words.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
        
chars = set(' abcdefghijklmnopqrstuvwxyz0123456789()?@？!$,~:.。，-_&=/􀆿\r\n')
for c in chars:
    stopwordset.add(c)

Building prefix dict from /home/tp6han/gp_new/jieba_txt/dict.txt.big.txt ...
Loading model from cache /tmp/jieba.u0ddb98113e9461fac3a7658d9e24e45e.cache
Loading model cost 1.489 seconds.
Prefix dict has been built succesfully.


In [11]:
# stopwordset

In [12]:
from gensim import models
import gensim
sentences = models.word2vec.LineSentence("corpusSegDone.txt")
model = models.word2vec.Word2Vec(sentences, size=1, window=5, min_count=5, workers=1)

In [13]:
length = len(grouped_data)
length

12691

In [14]:
top10_vector = np.zeros(shape=(length,10))
jieba.set_dictionary('jieba_txt/dict.txt.big.txt')
jieba.load_userdict("jieba_txt/userdict.txt")
for i in range(l):
    frequency = defaultdict(int)
    for j in range(len(grouped_data.iloc[i][1])):
        seglist = jieba.cut((grouped_data.iloc[i][1])[j])
        texts = [word for word in seglist if word not in stopwordset]
        for text in texts:
            frequency[text] += 1
    a = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True)
#     print(a)
    
    row = np.zeros(shape=(1,10))
    
    r = 10 if len(a) >= 10 else len(a)
    for x in range(r):
        if a[x][0] in model:
            show = float(model[a[x][0]])
            row[0][x] = show
    top10_vector[i] = row

Building prefix dict from /home/tp6han/gp_new/jieba_txt/dict.txt.big.txt ...
Loading model from cache /tmp/jieba.u0ddb98113e9461fac3a7658d9e24e45e.cache
Loading model cost 1.697 seconds.
Prefix dict has been built succesfully.


In [15]:
top10_vector.shape


(12691, 10)

# k-means clustering

In [16]:
from sklearn import cluster, datasets

# KMeans 演算法
kmeans_fit = cluster.KMeans(n_clusters = 3,random_state=1170).fit(top10_vector)

# 印出分群結果
cluster_labels = kmeans_fit.labels_
print("分群結果 前10：")
print(cluster_labels[:10])
print("---")

分群結果 前10：
[0 2 2 2 0 2 2 0 0 2]
---


In [17]:
len(cluster_labels)

12691

In [18]:
unique, counts = np.unique(cluster_labels, return_counts=True)
dict(zip(unique, counts))

{0: 6029, 1: 2426, 2: 4236}

In [19]:
t = grouped_data
t['category'] = cluster_labels

In [20]:
t = t.sort_values(by='category')
# t = t.drop('from',1)

In [21]:
t = t.reset_index(drop=True)
t.head()

,from,text,category
0,1000080793444348,"[嗨嗨, 安安, 幾歲呢, 我也是, 😂😂, 我男生, 嗨嗨, 我男生😂😂, 嗨嗨, 欸欸不...",0
1,1198976930175446,"[Hi 晚安, 這東西是新流行的嗎, 剛在迪卡上看到, 哈哈哈 開發者還有在下面留言 超酷,...",0
2,961303467312994,"[嗨, 有人嗎, 嗨, 真的有人哦😂, 讓我猜猜, 你是女生嗎, 😳, 是還是不是, 好啊,...",0
3,961304217331916,"[嗨, 嗨, 第一次用這個, 嗨, 剛剛用這個 好像有點複雜, 感覺訊息有點慢慢的, 喔 還...",0
4,1198852076843853,"[嗨, 呃, 我不知道, 我正想問你, 😅, 對啊！今天看到的😳, 我也是, 你也是?, 哈...",0


In [22]:
t.to_csv('../data/k_means_result.csv',encoding='utf-8-sig')

In [23]:
aaa = t.drop('from',1)
aaa = aaa.groupby('category')
# aaa

In [24]:
aaa = aaa.agg({'text': 'sum'})

### todo
* 沒講過超過幾句話的不納入計算 ex: 某使用者講不超過10句 就刪
* user 講幾句話的長條圖
* 同一類的使用者對話的比數是否高於其他不同累類的人聊天


In [63]:
lll = df.groupby('from')['text'].agg(['count']).reset_index()


In [64]:
lll[:3]
lll['count'].head()

0    81
1    40
2    15
3    16
4    60
Name: count, dtype: int64

In [72]:
n = pd.concat([df, lll['count']], axis=1)

In [85]:
n[n['count']>30].head()

,from,to,text,room,time,count
0,ud4097fbb4471c8c54512814bf0182e03,u84fa1e46e99df373e0f978c2ba274f9a,wd,8aee77a7-1d90-4d0f-b3c3-3f7b94a90e36,1.471640e+12,81.0
1,ud4097fbb4471c8c54512814bf0182e03,u96e826dccd3c9db56d684a5ddffc3154,嗨嗨,fe214cfc-22d1-4736-92e7-d133190142b8,1.471643e+12,40.0
4,u96e826dccd3c9db56d684a5ddffc3154,ud4097fbb4471c8c54512814bf0182e03,哈哈哈哈哈哈哈,fe214cfc-22d1-4736-92e7-d133190142b8,1.471643e+12,60.0
5,u96e826dccd3c9db56d684a5ddffc3154,ud4097fbb4471c8c54512814bf0182e03,這是限定版嗎,fe214cfc-22d1-4736-92e7-d133190142b8,1.471643e+12,329.0
6,ud4097fbb4471c8c54512814bf0182e03,u96e826dccd3c9db56d684a5ddffc3154,什麼限定版？,fe214cfc-22d1-4736-92e7-d133190142b8,1.471643e+12,82.0


In [86]:
q = df.groupby('room')['text'].agg(['count']).reset_index()
q.columns = ['room', 'room_dialog_count']
len(q)

39423

In [87]:
q.head()

,room,room_dialog_count
0,0000d3d1-cd74-4c6a-8ab6-8c9c0df594aa,2
1,000129e0-e9c9-4e51-8b7a-b782d220894e,1
2,0002fc8b-0951-47b0-9348-bf7decae9d93,6
3,000418d0-779f-469a-ad5b-7fcfd6da4959,26
4,00054188-59bc-49cf-aea2-e8ddd7057849,162


In [91]:
q.room_dialog_count.mean()

25.364913882758795